In [1]:
import random
import numpy as np
import pandas as pd

# primeiramente, crio três conjuntos de dados, sendo os dois primeiros, 'df0' e 'df1', casos linearmente 
# separáveis, e 'df2', não lineamente separável

# dados linearmente separáveis
df0 = pd.DataFrame([[0, 0, 0, 0], 
                    [1, 0, 0, 0],
                    [1, 1, 0, 1],
                    [1, 0, 1, 1],
                    [1, 1, 1, 1]],
                    columns = ['x0', 'x1', 'x2', 'y'])

# dados linearmente separáveis
df1 = pd.DataFrame([[0, 0, 0, 0, 0],
                    [0, 0, 1, 0, 0],
                    [1, 0, 0, 0, 0],
                    [1, 1, 0, 0, 0],
                    [1, 0, 1, 0, 0],
                    [1, 1, 1, 0, 1],
                    [1, 0, 1, 1, 1],
                    [1, 1, 1, 1, 1]],
                    columns = ['x0', 'x1', 'x2', 'x3', 'y'])

# dados NÃO linearmente separáveis
df2 = pd.DataFrame([[0, 0, 0, 1],
                    [0, 1, 0, 0],
                    [0, 0, 1, 0],
                    [1, 0, 1, 0],
                    [1, 1, 1, 1]],
                    columns = ['x0', 'x1', 'x2', 'y'])

# semente (para obtenção de coeficientes 'iniciais')
seed = 123
random.seed(seed)

In [2]:
# a função 'perceptron()' é a que, de fato, trabalha como um perceptron, as funções 'get_errors()' e 
# 'coefficient_optimization()' são executadas em 'loop', pela função 'perceptron()'

# a seguinte função desencadeia o primeiro processo empregado no perceptron; a obtenção de um y-chapéu para cada
# observação, através do somatório de coeficientes multiplicados pelas suas respectivas variáveis, em seguida,
# é feita a limiarização deste valor (função de ativação), os dois valores, limiarizado e não limiarizado são
# são armzenados (o valor limiarizado é comparado ao valor verdadeiro, já o valor não limiarizado é utilizado
# no processo de otimização) para cada observação, acompanhados do valor verdadeiro, para que, posteriormente, 
# possamos 'rastrear' erros e acertos (pelo índice da observação, que também é armazenado, por este motivo, 
# é importante que o dataframe contendo os dados tenha seu índice ordenado e possua um 'range' iniciando em 0,
# com 'step' de 1), o retorno da função consiste em três variáveis, sendo a primeira, o índice da primeira
# observação prevista incorretamente (em vez de implementar uma condição de parada 'instantânea', obtenho todas
# as predições e depois encontro a primeira, fiz o processo desta forma porque me pareceu mais fácil, por mais
# que resulte em trabalho desnecessário), no caso de todas as previsões estarem corretas, esta variável é 
# 'None', além disso a função também retorna os coeficientes (coefficients) e o dicinário (pred_dict) contendo
# as predições e valores esperados; estes são algo como 'variáveis globais', que sofrem consultas e alterações, 
# ao decorrer do processo
def get_errors(feature_matrix, label, coefficients, threshold, upper, lower): 
    
    '''Params:
       feature_matrix -> variáveis explicativas
       label -> valores verdadeiros
       coefficients -> vetor de coeficientes
       threshold -> limiar da função de ativação
       upper -> limite superior (valor assumido por y-chapéu, caso esteja acima de 'threshold')
       lower -> limite inferior (valor assumido por y-chapéu, caso esteja abaixo de 'threshold')'''
    
    count = 0
    pred_dict = {}
    for n in range(len(coefficients)): # calculando e limiarizando predições 
        predicted = round(sum(feature_matrix.iloc[n] * coefficients))
        
        if predicted >= threshold:
            pred_aux = upper
        else:
            pred_aux = lower
        
        pred_dict[count] = {'predicted' : predicted, 'actual' : label.iloc[n], 'pred_aux': pred_aux}
        count += 1
    
    # comparando predições limiarizadas com valores verdadeiros        
    error_ids = [key for key in pred_dict.keys() if pred_dict[key]['pred_aux'] != pred_dict[key]['actual']]
        
    if len(error_ids) == 0: # condição de parada (convergência)
        return None, coefficients, pred_dict      
    else:
        return min(error_ids), coefficients, pred_dict 

# esta função ajusta os coeficientes, de acordo com a primeira previsão incorreta, da última época, aqui
# também é feita a interpretação de uma das variáveis retornadas pela função anterior, com base nisso, a função
# retorna a condição de parada da próxima função (que realmente executa o 'loop'; esta função, da mesma forma 
# que a anterior, é uma etapa 'intermediária'), nesta função, introduzimos 'learning_rate', entretanto, não
# 'alimentamos' esta função diretamente, definimos o parâmetro na função seguinte, que engloba esta e a anterior
def coefficient_optimization(error_id, coef_list, pred_dict, learning_rate, feature_matrix, label): 
    
    if error_id == None: # interpreta condição de parada implementada retornada pela função anterior
        return 'stop', coef_list

    bad_row = feature_matrix.iloc[error_id] # <- aqui, acesso cada variável explicativa da observação, cuja 
                                            #    previsão falhou
    
    # a seguinte lista passará a ser o novo vetor de coeficientes, estes obtidos através do ajuste dos
    # ceoficientes anteriores, com base na primeira predição incorreta
    new_coefs = [coef_list[n]+learning_rate*bad_row[n]*(label.iloc[error_id]-pred_dict[error_id]['predicted'])\
                  for n in range(len(bad_row))]
    # coef_list[n] <- coeficiente, acessado 'posicionalmente', no vetor
    # bad_row[n] <- representa a variável explicativa da observação
    # (label.iloc[error_id] - pred_dict[error_id]['predicted']) <- subtração do valor previsto, não limiarizado,
                                                                #  do valor verdadeiro
                
    return 'go', new_coefs

def perceptron(feature_matrix, label, coefficients, learning_rate, threshold, upper, lower, max_epoch):
    
    run = 'go'
    epoch = 0
    while True:
        if run == 'go':
            
            error_id, coef_list, pred_dict = get_errors(feature_matrix, label, coefficients, 
                                                        threshold, upper, lower)
            
            run, new_coefs = coefficient_optimization(error_id, coef_list, pred_dict,
                                                     learning_rate, feature_matrix, label) 
            coefficients = new_coefs # redefinindo coeficientes
            epoch += 1
            # condição de parada (por convergência ou exaustão)
            if epoch == max_epoch and run != 'stop':
                return new_coefs, 'max_epoch reached, no convergence'

            elif epoch != max_epoch and run == 'stop':  
                return new_coefs, 'convergence achieved'

            elif epoch == max_epoch and run == 'stop':
                return new_coefs, 'max_epoch reached, convergence achieved'
            
# à partir daqui, exemplos de uso da função 'perceptron()' (em 'df0', 'df1', 'df2')
max_epoch = 25
learning_rate = 0.1
threshold = 0.5
upper = 1
lower = 0

In [3]:
# ajuste a 'df0' (linearmente separável)

# coeficientes de df0 não otimizados (gerados 'aleatoriamente')
coefficients0 = random.choices(population = np.arange(start = -1, stop = 1, step = 0.001), k = 3)
print(f'previous coefficients: {coefficients0}')

coefs0, report0 = perceptron(coefficients = coefficients0, threshold = threshold, upper = upper, 
                             lower = lower, learning_rate = learning_rate,
                             feature_matrix = df0[['x0', 'x1', 'x2']], label = df0['y'],
                             max_epoch = max_epoch)

print(f'updated coefficients:  {coefs0}')
print(f'report:                {report0}')

previous coefficients: [-0.8959999999999999, -0.8259999999999998, -0.18599999999999928]
updated coefficients:  [0.30400000000000016, 0.3740000000000002, -0.18599999999999928]
report:                convergence achieved


In [4]:
# ajuste a 'df1' (linearmente separável)

# coeficientes de df1 não otimizados (gerados 'aleatoriamente')
coefficients1 = random.choices(population = np.arange(start = -1, stop = 1, step = 0.001), k = 4)
print(f'previous coefficients: {coefficients1}')

coefs1, report1 = perceptron(coefficients = coefficients1, threshold = threshold, upper = upper,
                             lower = lower, learning_rate = learning_rate,
                             feature_matrix = df1[['x0', 'x1', 'x2', 'x3']], 
                             label = df1['y'], max_epoch = max_epoch)

print(f'updated coefficients:  {coefs1}')
print(f'report:                {report1}')

# não foram necessários ajustes, mas alterando a 'seed', estes coeficientes, muito provavelmente serão 
# (testei as seeds 777 e 9000, ambas resultam em ajuste sutil no terceiro coeficiente, a seed 0 também 
# resulta em um conjunto de coeficientes que não necessita de ajustes)

previous coefficients: [-0.7849999999999998, 0.8020000000000016, -0.9239999999999999, 0.07200000000000095]
updated coefficients:  [-0.7849999999999998, 0.8020000000000016, -0.9239999999999999, 0.07200000000000095]
report:                convergence achieved


In [5]:
# ajuste a df2 (NÃO linearmente separável)

# coeficientes de df2 não otimizados (gerados 'aleatoriamente')
coefficients2 = random.choices(population = np.arange(start = -1, stop = 1, step = 0.001), k = 3)
print(f'previous coefficients: {coefficients2}')

coefs2, report2 = perceptron(coefficients = coefficients2, threshold = threshold, upper = upper,
                             lower = lower, learning_rate = learning_rate, 
                             feature_matrix = df2[['x0', 'x1', 'x2']], label = df2['y'],
                             max_epoch = max_epoch)

print(f'updated coefficients:  {coefs2}')
print(f'report:                {report2}')

# não otimiza

previous coefficients: [-0.3359999999999994, 0.7040000000000015, -0.6809999999999997]
updated coefficients:  [-0.3359999999999994, 0.7040000000000015, -0.6809999999999997]
report:                max_epoch reached, no convergence
